In [1]:
import pandas as pd

df = pd.read_csv("../data/DSNY_Monthly_Tonnage_Data.csv")
df.head

<bound method NDFrame.head of            MONTH        BOROUGH  COMMUNITYDISTRICT  REFUSETONSCOLLECTED  \
0      2025 / 10          Bronx                  1               3352.0   
1      2025 / 10          Bronx                  2               7942.4   
2      2025 / 10          Bronx                  3               2348.6   
3      2025 / 10          Bronx                  4               4210.4   
4      2025 / 10          Bronx                  5               3578.3   
...          ...            ...                ...                  ...   
24642  1990 / 06  Staten Island                  2               3485.8   
24643  1990 / 06  Staten Island                  3               4242.2   
24644  1990 / 01      Manhattan                  3                 24.4   
24645  1990 / 01         Queens                 12                  8.4   
24646  1990 / 01  Staten Island                  2                 39.1   

       PAPERTONSCOLLECTED  MGPTONSCOLLECTED  RESORGANICSTONS  \
0    